# Labelling crawled data with senticnet API

In [3]:
!pip install senticnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 MB 4.0 MB/s eta 0:00:00


In [19]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 3.1 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16677 sha256=9aa7c8d1b3217589dfb7c148364c21ec699c56f5f044c81cdff3a96fc96471b1
  Stored in directory: /root/.cache/pip/wheels/41/01/29/deaa71fe596f8d857e57c4fb388db8861e23e6ed0b03204dcb
Successfully built pandarallel


In [4]:
import pandas as pd
from senticnet.senticnet import SenticNet
import requests

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk 
import re

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import seaborn as sb
import sklearn
import spacy
import re
!pip install gensim pyldavis wordcloud textblob textstat

nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
imdbDF = pd.read_csv('/content/drive/MyDrive/final_cleaned_tmdb_data.csv.csv')
imdbDF.head()

,Unnamed: 0,Movie Name,Movie Id,Review Content,Sentiment,tmdb Rating,User Rating,Genre(s),Not A Movie?,Release Date,Production Company
0,0,Shadows in Paradise,3,"Released in 1986, Aki Kaurismaki's <i>Varjoja ...",Positive,7.184,7.0,"Drama,Comedy,Romance",False,1986-10-17,Villealfa Filmproductions
1,1,American Beauty,14,The film American Beauty to me is a film about...,Positive,8.030,10.0,Drama,False,1999-09-15,Jinks/Cohen Company
2,2,Citizen Kane,15,"""The Greatest Movie of All Time!"" That's the g...",Positive,8.023,9.0,"Mystery,Drama",False,1941-04-17,Mercury Productions
3,3,Dancer in the Dark,16,Dazzling Björk in this indictment against the ...,Positive,7.889,10.0,"Drama,Crime",False,2000-06-30,Lantia Cinema & Audiovisivi
4,4,The Fifth Element,18,This movie is one of the best movies of all ti...,Positive,7.524,9.5,"Adventure,Fantasy,Action,Thriller,Science Fiction",False,1997-05-02,Gaumont


In [10]:
imdbDF = imdbDF[['Review Content', 'Sentiment']]
imdbDF.head()

,Review Content,Sentiment
0,"Released in 1986, Aki Kaurismaki's <i>Varjoja ...",Positive
1,The film American Beauty to me is a film about...,Positive
2,"""The Greatest Movie of All Time!"" That's the g...",Positive
3,Dazzling Björk in this indictment against the ...,Positive
4,This movie is one of the best movies of all ti...,Positive


## Preprocessing func

In [14]:
default_stopwords = set(nltk.corpus.stopwords.words('english'))
#excluding some useful words from stop words list as we doing sentiment analysis
excluding = set(['against','not','don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
             'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
             'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",'shouldn', "shouldn't", 'wasn',
             "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"])

stopword = default_stopwords - excluding

# create a lemmatizer object
lemmatizer = WordNetLemmatizer()

In [12]:
# Remove any HTML tags
def clean_html(sentence): 
    pattern = re.compile('<.*?>')
    cleaned_text = re.sub(pattern,' ',sentence)
    return cleaned_text

#Function to keep only words containing letters A-Z and a-z.
#this will remove all punctuations, special characters.
def rem_pun(sentence):
    cleaned_text  = re.sub('[^a-zA-Z]',' ',sentence)
    return (cleaned_text)

#Remove URL from sentences.
def rem_url(sen):
    txt = re.sub(r"http\S+", " ", sen)
    sen = re.sub(r"www.\S+", " ", txt)
    return (sen)

#Remove words like 'ddddddddd', 'funnnnnn', 'coolllllll' etc. Preserves words like 'goods', 'cool', 'best' etc. We will remove all such words which has three consecutive repeating characters.
def remove_extra(sen): 
    cleaned_text  = re.sub("\s*\b(?=\w*(\w)\1{2,})\w*\b",' ',sen)
    return (cleaned_text)

def remove_stopword_lemmatize(sen):
    text_tokenized = re.split('[^(A-Za-z0-9_$)]+', sen) # tokenization, remove non-words
    text_nonstop = [lemmatizer.lemmatize(word) for word in text_tokenized if word not in stopword] # remove stopwords and stemming
    return " ".join(text_nonstop)

def lower_case(x):
    x = str(x).lower()
    x = x.replace(",000,000", " m").replace(",000", " k").replace("′", "'").replace("’", "'")\
                           .replace("won't", " will not").replace("cannot", " can not").replace("can't", " can not")\
                           .replace("n't", " not").replace("what's", " what is").replace("it's", " it is")\
                           .replace("'ve", " have").replace("'m", " am").replace("'re", " are")\
                           .replace("he's", " he is").replace("she's", " she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("how's"," how has").replace("y'all"," you all")\
                           .replace("o'clock"," of the clock").replace("ne'er"," never").replace("let's"," let us")\
                           .replace("finna"," fixing to").replace("gonna"," going to").replace("gimme"," give me").replace("gotta"," got to").replace("'d"," would")\
                           .replace("daresn't"," dare not").replace("dasn't"," dare not").replace("e'er"," ever").replace("everyone's"," everyone is")\
                           .replace("'cause'"," because")
    
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    return x

In [15]:
#Cleaning the text from dataset
def cleaned_text(review):
    review = lower_case(review)
    review = clean_html(review)
    review = rem_url(review)
    review = rem_pun(review)
    review = remove_extra(review)
    review = remove_stopword_lemmatize(review)
    return review
  
imdbDF['text']=imdbDF['Review Content'].apply(cleaned_text)
imdbDF.head()

,Review Content,Sentiment,text
0,"Released in 1986, Aki Kaurismaki's <i>Varjoja ...",Positive,released aki kaurismaki varjoja paratiisissa s...
1,The film American Beauty to me is a film about...,Positive,film american beauty film purpose purpose life...
2,"""The Greatest Movie of All Time!"" That's the g...",Positive,greatest movie time general opinion movie agr...
3,Dazzling Björk in this indictment against the ...,Positive,dazzling bj rk indictment against death senten...
4,This movie is one of the best movies of all ti...,Positive,movie one best movie time great story line gra...


## Load API

In [16]:
apiURL = "http://sentic.net/api/en/zfCm4Hq4p0lgOJNU.py?text="
response = requests.get(apiURL+"Real-time trading alerts from our team")
print(response.content.decode('ASCII'))

POSITIVE



## Load dataset and preprocess

In [17]:
subset = imdbDF.sample(frac=0.1)
subset

,Review Content,Sentiment,text
6080,Henry Fonda is adequate but really nothing mor...,NaN,henry fonda adequate really nothing story th c...
5529,The Cross Guns. Colt .45 (AKA: Thundercloud) ...,NaN,cross gun colt aka thundercloud directed edwin...
517,"***The Passion of the Christ, I mean Superman*...",Positive,passion christ mean superman superman return ...
7911,There is a particular moment in Nimirnthu Nil ...,NaN,particular moment nimirnthu nil stop taking fi...
1033,Not done with the 1st month of 2017 when I saw...,NaN,not done st month saw officially first new fav...
...,...,...,...
1189,I have to admit up front to liking every singl...,NaN,admit front liking every single john carpenter...
4296,"Bad watch, probably won't watch again, and can...",NaN,bad watch probably not watch not recommend man...
8869,Holy Cow....seriously. I try to be unbiased b...,NaN,holy cow seriously try unbiased movie four sha...
4483,This was an excellent film and it really surpr...,NaN,excellent film really surprised included cheap...


In [ ]:
subset['Polarity'] = subset["text"].apply(lambda x: requests.get(apiURL + x).content.decode('ASCII'))

In [21]:
from pandarallel import pandarallel
 
pandarallel.initialize(progress_bar=True, nb_workers=8)

def my_func(text):
    import requests
    apiURL = "http://sentic.net/api/en/jFz4p0loUTjnz.py?text="
    return requests.get(apiURL + text).content.decode('ASCII')

subset['Polarity'] = subset["text"].parallel_apply(my_func)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [22]:
subset["Polarity"].value_counts()

Your free trial has expired: to continue using your APIs, please contact <a href="mailto:business@sentic.net">business@sentic.net</a>\n                                                                                                                              988
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>414 Request-URI Too Long</title>\n</head><body>\n<h1>Request-URI Too Long</h1>\n<p>The requested URL's length exceeds the capacity\nlimit for this server.<br />\n</p>\n</body></html>\n      1
Name: Polarity, dtype: int64

## TextBlob

In [23]:
import textblob 
from textblob import TextBlob

In [24]:
# function to calculate polarity 
def getPolarity(review): 
    return TextBlob(review).sentiment.polarity 
 
# function to analyze the reviews 
def polarity_analysis(score): 
    if score == 0: 
        return 'Neutral' 
    elif score > 0: 
        return 'Positive' 
    else: 
        return 'Negative'

In [25]:
subset['Polarity'] = subset['text'].parallel_apply(getPolarity)  
subset['Polarity'] = subset['Polarity'].parallel_apply(polarity_analysis)

In [27]:
subset = subset[['text', 'Polarity']]

In [28]:
subset.head()

,text,Polarity
6080,henry fonda adequate really nothing story th c...,Positive
5529,cross gun colt aka thundercloud directed edwin...,Positive
517,passion christ mean superman superman return ...,Positive
7911,particular moment nimirnthu nil stop taking fi...,Negative
1033,not done st month saw officially first new fav...,Positive


In [29]:
subset.to_csv('subset_labelled.csv', index = False)